## Regularization

#### Table of contents

* [Overview](#Overview)
* [Setup](#Setup)
* [Lasso](#Lasso)

# Overview

This script covers how to implement two techniques:
1. Regularization
2. Cross-validation

Regularization is a shrinkage estimator. 
By adding an additional constraint on the optimization problem, the coefficients will be smaller than their non-constrained counterparts.
With an absolute constraint, it can be used to **select** features by setting coefficients to zero.
The shrinkage strength is determined by a hyperpameter $\lambda$, where $\lambda = 0$ is no shrinkage.

Cross-validation is a ubiquitous machine learning technique used **tune** hyperparameters.
Where train-test splits are used to compare across model classes, cross-validation is used for comparing within model classes.
See the lecture slides for more details.

***
# Setup
[TOP](#Regularization)

In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import linear_model as lm

In [13]:
df = pd.read_pickle('C:/Users/hubst/Econ490_group/class_data.pkl')
df.columns

Index(['GeoName', 'pct_d_rgdp', 'urate_bin', 'pos_net_jobs', 'emp_estabs',
       'estabs_entry_rate', 'estabs_exit_rate', 'pop', 'pop_pct_black',
       'pop_pct_hisp', 'lfpr', 'density', 'year'],
      dtype='object')

In [14]:
df_prepped = df.drop(columns = ['urate_bin', 'year', 'GeoName']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])

In [15]:
y = df_prepped['pct_d_rgdp']
x = df_prepped.drop(columns = 'pct_d_rgdp')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)

###################################################
# Be careful if you have a large data set.        #
# We have created ~4 copies of the original data: #
#                                                 #
# 0. df                                           #
# 1. df_prepped                                   #
# 2. x, y                                         #
# 3. x_train, x_test, y_train, y_test             #
# 4. x_train_std, x_test_std                      #
#                                                 #
# you may run out of memeory                      #
###################################################

***
# Lasso
[TOP](#Regularization)

`statsmodels` fits an elastic-net constraint, which is a hybrid of Lasso and Ridge regression. 
It has the penalty:
$$
\alpha \left[ (1-L1\_wt)||\beta||_2 + L1\_wt||\beta||_1 \right]
$$

In [16]:
fit_ridge = sm.OLS(y_train, x_train_std).fit_regularized(alpha = 10, L1_wt = 0)
# note: there is no fit.summary()
fit_ridge.params

array([ 0.1764772 ,  0.07331063, -0.01526529,  0.08336556, -0.01482107,
       -0.00042219, -0.0357732 ,  0.03635522,  0.06818012, -0.00205087,
        0.04915225, -0.01196064,  0.02367039,  0.02543083,  0.01612287,
        0.0579132 , -0.01519722, -0.02717455, -0.06363553,  0.02413141,
       -0.00177043, -0.01661271,  0.0106122 , -0.01318967, -0.00675205,
       -0.03770638, -0.01219879,  0.00823161])

In [17]:
fit_lasso = sm.OLS(y_train, x_train_std).fit_regularized(alpha = 10, L1_wt = 1)
# note: there is no fit.summary()
fit_lasso.params

const                0.0
pos_net_jobs         0.0
emp_estabs           0.0
estabs_entry_rate    0.0
estabs_exit_rate     0.0
pop                  0.0
pop_pct_black        0.0
pop_pct_hisp         0.0
lfpr                 0.0
density              0.0
lower                0.0
similar              0.0
2003                 0.0
2004                 0.0
2005                 0.0
2006                 0.0
2007                 0.0
2008                 0.0
2009                 0.0
2010                 0.0
2011                 0.0
2012                 0.0
2013                 0.0
2014                 0.0
2015                 0.0
2016                 0.0
2017                 0.0
2018                 0.0
dtype: float64

***
# Cross-Validation 
[TOP](#Regularization)

We are using a specific type of k-fold cross-validation: grid search.
When performing grid search cross-validation, you always want to ensure that your selected hyperparameters are an interior point of the grid. 
Otherwise, you have no selected the optimal solution.

The grid search function is from `sklearn`.
Consequently, we need to use functions from the `linear_model` module: https://scikit-learn.org/stable/modules/linear_model.html.

In [18]:
param_grid = [
    {'alpha': 10**np.linspace(-2, 2, num = 20)}
]

# We are manually supplying an intercept
# and standardized (not normalized) the features
cv_lasso = lm.Lasso(fit_intercept = False, normalize = False,
                    random_state = 490)
grid_search = GridSearchCV(cv_lasso, param_grid, cv = 5,
                         scoring = 'neg_root_mean_squared_error')
grid_search.fit(x_train_std, y_train)
print(grid_search.best_params_)
best = grid_search.best_params_['alpha']
best

{'alpha': 0.016237767391887217}


0.016237767391887217

This is an interior point ($10^{-2} = 0.01$). 
We need to adjust our grid search.

In [19]:
param_grid = [
    {'alpha': 10**np.linspace(-5, -2, num = 20)}
]

cv_lasso = lm.Lasso(fit_intercept = False, normalize = False,
                    random_state = 490)

grid_search = GridSearchCV(cv_lasso, param_grid, cv = 5,
                         scoring = 'neg_root_mean_squared_error')
grid_search.fit(x_train_std, y_train)
best = grid_search.best_params_['alpha']
best

0.01

***
Once we have identified the best value of $\alpha$, then the next step is to estimate the full model. 
We will use `statsmodels` for this task.

In [20]:
fit_lasso_tuned = sm.OLS(y_train, x_train_std).fit_regularized(L1_wt = 1, alpha = best)
fit_lasso_tuned.params

const                1.931249
pos_net_jobs         0.493323
emp_estabs          -0.114711
estabs_entry_rate    0.766665
estabs_exit_rate    -0.384190
pop                 -0.100041
pop_pct_black        0.000000
pop_pct_hisp         0.383635
lfpr                 0.524511
density             -0.027861
lower                0.505713
similar              0.175849
2003                 0.055653
2004                 0.058518
2005                 0.076016
2006                 0.436149
2007                -0.319301
2008                -0.349027
2009                -0.591098
2010                 0.236479
2011                 0.000000
2012                -0.280321
2013                 0.010085
2014                -0.264897
2015                -0.208615
2016                -0.527701
2017                -0.216195
2018                 0.000000
dtype: float64

***
Remember, a regularized model is biased.
We will fit 

- a non-regularized standardized model excluding the features that are zero 
- a non-regularized, non-standardized model excluding the features that are zero

In [21]:
beta = fit_lasso_tuned.params
beta.index[beta == 0]

Index(['pop_pct_black', 2011, 2018], dtype='object')

In [22]:
x_train_std_trim = x_train_std.loc[:, ~x_train_std.columns.isin(beta.index[beta == 0])]
x_test_std_trim = x_test_std.loc[:, ~x_test_std.columns.isin(beta.index[beta == 0])]

In [23]:
fit_std_final = sm.OLS(y_train, x_train_std_trim).fit()
fit_std_final.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.040      
Dependent Variable: pct_d_rgdp       AIC:                241644.5897
Date:               2021-03-03 18:35 BIC:                241855.0110
No. Observations:   33418            Log-Likelihood:     -1.2080e+05
Df Model:           24               F-statistic:        59.67      
Df Residuals:       33393            Prob (F-statistic): 1.40e-281  
R-squared:          0.041            Scale:              80.830     
--------------------------------------------------------------------
                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const                1.9412   0.0492  39.4717 0.0000  1.8449  2.0376
pos_net_jobs         0.4985   0.0531   9.3944 0.0000  0.3945  0.6025
emp_estabs          -0.1244   0.0525  -2.3694 0.0178 -0.2273 -0.0215
estabs_entry_rate    0.7741   0.0587  13.1872 0.0000  0.6590  0.8891
estabs_exit_rate    -0.4005   0.0570  -7.0309 0.0000 -0.5122 -0.2889
pop                 -0.1080   0.0551  -1.9618 0.0498 -0.2159 -0.0001
pop_pct_hisp         0.3970   0.0507   7.8255 0.0000  0.2976  0.4964
lfpr                 0.5229   0.0573   9.1268 0.0000  0.4106  0.6352
density             -0.0351   0.0524  -0.6688 0.5036 -0.1378  0.0677
lower                0.5305   0.0667   7.9595 0.0000  0.3999  0.6611
similar              0.2014   0.0574   3.5072 0.0005  0.0888  0.3139
2003                 0.0584   0.0556   1.0517 0.2930 -0.0505  0.1673
2004                 0.0604   0.0556   1.0867 0.2772 -0.0486  0.1694
2005                 0.0868   0.0590   1.4719 0.1411 -0.0288  0.2024
2006                 0.4474   0.0605   7.3987 0.0000  0.3289  0.5660
2007                -0.3358   0.0557  -6.0277 0.0000 -0.4449 -0.2266
2008                -0.3631   0.0554  -6.5538 0.0000 -0.4717 -0.2545
2009                -0.6025   0.0564 -10.6850 0.0000 -0.7130 -0.4920
2010                 0.2417   0.0555   4.3524 0.0000  0.1328  0.3505
2012                -0.2962   0.0553  -5.3534 0.0000 -0.4047 -0.1878
2013                 0.0130   0.0553   0.2356 0.8137 -0.0954  0.1215
2014                -0.2822   0.0556  -5.0786 0.0000 -0.3911 -0.1733
2015                -0.2264   0.0555  -4.0793 0.0000 -0.3352 -0.1176
2016                -0.5459   0.0556  -9.8234 0.0000 -0.6548 -0.4370
2017                -0.2337   0.0556  -4.2066 0.0000 -0.3426 -0.1248
--------------------------------------------------------------------
Omnibus:            34262.381     Durbin-Watson:        1.988       
Prob(Omnibus):      0.000         Jarque-Bera (JB):     11837132.892
Skew:               4.477         Prob(JB):             0.000       
Kurtosis:           94.766        Condition No.:        3           
====================================================================

"""

In [24]:
x_train_trim = x_train.loc[:, ~x_train.columns.isin(beta.index[beta == 0])]
x_test_trim = x_test.loc[:, ~x_test.columns.isin(beta.index[beta == 0])]

In [25]:
fit_final = sm.OLS(y_train, x_train_trim).fit()
fit_final.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.040      
Dependent Variable: pct_d_rgdp       AIC:                241644.5897
Date:               2021-03-03 18:35 BIC:                241855.0110
No. Observations:   33418            Log-Likelihood:     -1.2080e+05
Df Model:           24               F-statistic:        59.67      
Df Residuals:       33393            Prob (F-statistic): 1.40e-281  
R-squared:          0.041            Scale:              80.830     
--------------------------------------------------------------------
                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const               -3.1782   0.5046  -6.2983 0.0000 -4.1673 -2.1891
pos_net_jobs         1.0046   0.1069   9.3944 0.0000  0.7950  1.2142
emp_estabs          -0.0265   0.0112  -2.3694 0.0178 -0.0485 -0.0046
estabs_entry_rate    0.2604   0.0197  13.1872 0.0000  0.2217  0.2990
estabs_exit_rate    -0.1583   0.0225  -7.0309 0.0000 -0.2025 -0.1142
pop                 -0.0000   0.0000  -1.9618 0.0498 -0.0000 -0.0000
pop_pct_hisp         0.0305   0.0039   7.8255 0.0000  0.0229  0.0382
lfpr                 0.0480   0.0053   9.1268 0.0000  0.0377  0.0583
density             -0.0000   0.0000  -0.6688 0.5036 -0.0001  0.0000
lower                1.0935   0.1374   7.9595 0.0000  0.8242  1.3628
similar              0.5213   0.1486   3.5072 0.0005  0.2300  0.8126
2003                 0.2469   0.2348   1.0517 0.2930 -0.2133  0.7071
2004                 0.2556   0.2352   1.0867 0.2772 -0.2054  0.7166
2005                 0.3685   0.2503   1.4719 0.1411 -0.1222  0.8591
2006                 1.8775   0.2538   7.3987 0.0000  1.3801  2.3749
2007                -1.4109   0.2341  -6.0277 0.0000 -1.8696 -0.9521
2008                -1.5361   0.2344  -6.5538 0.0000 -1.9955 -1.0767
2009                -2.5734   0.2408 -10.6850 0.0000 -3.0455 -2.1014
2010                 1.0264   0.2358   4.3524 0.0000  0.5642  1.4886
2012                -1.2668   0.2366  -5.3534 0.0000 -1.7306 -0.8030
2013                 0.0557   0.2363   0.2356 0.8137 -0.4075  0.5189
2014                -1.2035   0.2370  -5.0786 0.0000 -1.6679 -0.7390
2015                -0.9732   0.2386  -4.0793 0.0000 -1.4408 -0.5056
2016                -2.3327   0.2375  -9.8234 0.0000 -2.7981 -1.8672
2017                -0.9854   0.2342  -4.2066 0.0000 -1.4445 -0.5262
--------------------------------------------------------------------
Omnibus:            34262.381     Durbin-Watson:        1.988       
Prob(Omnibus):      0.000         Jarque-Bera (JB):     11837132.892
Skew:               4.477         Prob(JB):             0.000       
Kurtosis:           94.766        Condition No.:        3812400     
====================================================================
* The condition number is large (4e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

***
Lastly, we will test the models

In [26]:
rmse_null = np.sqrt(np.mean(  (y_test - np.mean(y_train))**2  ))
rmse_null

9.295172646932564

In [27]:
rmse_lasso = np.sqrt(np.mean(  (y_test - fit_lasso_tuned.predict(x_test_std))**2  ))
print(rmse_lasso)
round((rmse_lasso - rmse_null)/rmse_null*100, 2)

9.103321461021201


-2.06

In [28]:
rmse_std_final = np.sqrt(np.mean(  (y_test - fit_std_final.predict(x_test_std_trim))**2  ))
print(rmse_std_final)
round((rmse_std_final - rmse_null)/rmse_null*100, 2)

9.103231259163913


-2.06

In [29]:
rmse_final = np.sqrt(np.mean(  (y_test - fit_final.predict(x_test_trim))**2  ))
print(rmse_final)
round((rmse_final - rmse_null)/rmse_null*100, 2)

9.103358969607513


-2.06

Backward selection gave us a 1.37% reduction in RMSE. 

# What does ^ tell you about statistical significance for predictive power?